In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import json
from tqdm import tqdm
import os
import ast
import pickle
import pickle as pkl
import time
from abc import ABC
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aymer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aymer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importing the data

In [88]:
DATA_FOLDER = 'data/MovieSummaries/'
ADDITIONAL_FOLDER = 'data/AdditionalData/'

In [89]:
# reading a txt file and convert it to a dataframe
def read_txt(path):
    df = pd.read_csv(path, sep='\t', header=None)
    return df

In [90]:
# importing the data
plots = pd.read_csv(DATA_FOLDER + 'plot_summaries.txt',header=None, sep="\t")
movies = pd.read_csv(DATA_FOLDER + 'movie.metadata.tsv',header=None, sep="\t")
characters = pd.read_csv(DATA_FOLDER + 'character.metadata.tsv',header=None, sep="\t")
names = pd.read_csv(DATA_FOLDER + 'name.clusters.txt',header=None, sep="\t")
tvtropes = pd.read_csv(DATA_FOLDER + 'tvtropes.clusters.txt',header=None, sep="\t")

In [91]:
# renaming columns the plots dataframe
plots.columns = ['wikipedia_movie_id', 'plot']

`movies` data

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

In [92]:
# renaming the movie dataframe columns
movies.columns = ['wikipedia_movie_id', 'freebase_movie_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres']

`characters` data:
1. Wikipedia movie ID
2. Freebase movie ID
3. Movie release date
4. Character name
5. Actor date of birth
6. Actor gender
7. Actor height (in meters)
8. Actor ethnicity (Freebase ID)
9. Actor name
10. Actor age at movie release
11. Freebase character/actor map ID
12. Freebase character ID
13. Freebase actor ID

In [93]:
# fixing the character columns
characters.columns = ['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 'date_of_birth', 'gender', 'height', 'ethnicity', 'name', 'age_at_release', 'freebase_character_map_id', 'freebase_character_id', 'freebase_actor_id']

# Data preprocessing

In [94]:
revenue_nans = movies[movies['revenue'].isna()]["wikipedia_movie_id"]
print('Number of movies without revenue data: ', revenue_nans.shape[0])

Number of movies without revenue data:  73340


## Scraping

A lot of the movies have empty Box Office Revenue so one idea ot fix this is to acquire that data through other means.

Here we scraped the ImDB website to retrieve some of these informations about movie revenues as well as additional interesting informations such as movie ratings, movie producers...

In [95]:
import imdb_scraper as imdb

### Example on how to use the scraper

<b><span style="color:red">Don't close the browser when it is in use </span></b>

In [96]:
# initialize the scraper object
myscraper = imdb.ImdbScraper()

# select a movie wikipedia id
movie_id = movies.iloc[74623]["wikipedia_movie_id"]

# scrape the movie infos
scraped_data = myscraper.get_imdb_infos(movie_id)

# close the browser
myscraper.close()

scraped_data

{'global_revenue': None,
 'budget': 2846000.0,
 'gross_domestic': None,
 'opening_weekend': None,
 'rating_score': 6.7,
 'number_of_ratings': 1700.0,
 'watched_rank': None,
 'producer': 'Robert Wise',
 'release_year': None}

### Let the scraping begin!

Computing `n_computer` partitions to parallelize ImDB's scraping on multiple computers

In [97]:
n           = movies.shape[0]
n_computer  = 6
size        = n//n_computer

# create a uniform partition of indices from 0 to n-1 for n_computer computers
indices         = [i for i in range(n)]
partitions      = [indices[i*size:(i+1)*size] for i in range(n_computer)]
partitions[-1]  = partitions[-1] + indices[n_computer*size:] 

In [98]:
# show first and last element of each partition
partition_intervals = [(partitions[i][0], partitions[i][-1]) for i in range(n_computer)]
print(f"number of elements: {n}")
print(f"partitions' size: {size}\n")
for i in range(len(partition_intervals)):
    print(f"partition {i}: {partition_intervals[i]}")

number of elements: 81741
partitions' size: 13623

partition 0: (0, 13622)
partition 1: (13623, 27245)
partition 2: (27246, 40868)
partition 3: (40869, 54491)
partition 4: (54492, 68114)
partition 5: (68115, 81740)


Each user should use its index to compute his attributed dataset

In [99]:
"""
Select your index here:
    - Anthony   0 & 1
    - Anton     2
    - Aymeric   3
    - Eric      4
    - Yara      5
"""
index = 0

Since ImDB has a protection against bots, using `requests` yields a `forbidden` error. The use of `selenium` to simmulate a real human operator avoids this problem and allows to scrape, though more slowly.

In [100]:
def scrape_partition(index):
    """
    Scrape the IMDB data for the movies in the partition with the given index.
    The scraped dataset is saved in a separate csv file for each partition.

    Input:
    index(int) : The index of the partition to scrape.

    Output:
    None
    """

    # create a header dataframe
    header = pd.DataFrame(columns=["wikipedia_movie_id", "name", "global_revenue", "budget", "gross_domestic", "opening_weekend", 
                               "rating_score", "number_of_ratings", "watched_rank", "producer", "release_year"])  
    
    # csv file name
    csv_file = ADDITIONAL_FOLDER + 'imdb_partitions' + '/imdb_scraped_data_' + str(index) + '.csv'
    
    #initialize the scraper object
    myscraper = imdb.ImdbScraper()
    
    # if the csv_file doesn't exist, create it and write the header
    if not os.path.isfile(csv_file):
        header.to_csv(csv_file, index=False)
        starting_index = 0
    else:
        scraped = pd.read_csv(csv_file)
        starting_index = scraped.shape[0]
       
    index_range = partitions[index][starting_index:]
    
    for i in index_range:
        movie_id = movies.iloc[i]["wikipedia_movie_id"]
    
        movie_infos = myscraper.get_imdb_infos(movie_id)
        row = [movie_id, movies["name"].values[i], *list(movie_infos.values())]
    
        # create a new DataFrame for the row
        row_df = pd.DataFrame([row], columns=header.columns)
    
        # replace None values with the string 'None'
        row_df = row_df.fillna('None')
    
        # append the row DataFrame to the CSV file
        row_df.to_csv(csv_file, mode='a', header=False, index=False)
    
    myscraper.close()

In [101]:
# run the scraper
scrape_partition(index)

### Multi-thread version

This part shows an implementation of the scraper running in parallel on multiple threads on a single computer. The number of threads `n_threads` can be increased if the user's computer has access to more threads.

In [102]:
import threading

In [103]:
n           = movies.shape[0]
n_threads   = 6
size        = n//n_threads

# create a uniform partition of indices from 0 to n-1 for n_threads computers
indices         = [i for i in range(n)]
partitions      = [indices[i*size:(i+1)*size] for i in range(n_threads)]
partitions[-1]  = partitions[-1] + indices[n_threads*size:] 

In [104]:
# show first and last element of each partition
partition_intervals = [(partitions[i][0], partitions[i][-1]) for i in range(n_threads)]
print(f"number of elements: {n}")
print(f"partitions' size: {size}\n")
for i in range(len(partition_intervals)):
    print(f"partition {i}: {partition_intervals[i]}")

number of elements: 81741
partitions' size: 13623

partition 0: (0, 13622)
partition 1: (13623, 27245)
partition 2: (27246, 40868)
partition 3: (40869, 54491)
partition 4: (54492, 68114)
partition 5: (68115, 81740)


In [105]:
threads = []

for i in range(n_threads):
    threads.append(threading.Thread(target=scrape_partition, args=(i,)))

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

### Merging the scraped partitions

In [106]:
csv_file = ADDITIONAL_FOLDER + r"/imdb_scraped_dataset.csv"
n_computer  = 6
try:
    partition_dataset = pd.read_csv(csv_file)
except:
    partition_file = ADDITIONAL_FOLDER + r'imdb_partitions/imdb_scraped_data_'
    partition_dataset = pd.read_csv(partition_file + '0.csv')

    for i in range(1, n_computer):
        partition = partition_file + str(i) + '.csv'
        partition_dataset = pd.concat([partition_dataset, pd.read_csv(partition)], axis=0).fillna('None')
        partition_dataset.to_csv(csv_file, index=False)

### Completing missing revenue data

In [107]:
# reading the scraped dataset
imdb_movies = pd.read_csv(ADDITIONAL_FOLDER + 'imdb_scraped_dataset.csv')
imdb_movies.head(3)

,wikipedia_movie_id,name,global_revenue,budget,gross_domestic,opening_weekend,rating_score,number_of_ratings,watched_rank,producer,release_year
0,975900,Ghosts of Mars,14010832.0,28000000.0,8709640.0,3804452.0,4.9,57000.0,3850.0,John Carpenter,2001.0
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,None,None,None,None,6.0,69.0,None,Edward Lucas,2000.0
2,28463795,Brun bitter,None,None,None,None,5.6,40.0,None,Sølve Skagen,1988.0


In [108]:
# computing number of movies without global revenue data in the imdb dataset
global_revenue_nans = imdb_movies[imdb_movies['global_revenue'].isna()]["wikipedia_movie_id"]
print('Number of movies without global revenue data: ', global_revenue_nans.shape[0])

Number of movies without global revenue data:  0


We have less missing revenue data in the scraped data

In [109]:
# calculating number of movies that have missing revenue in the original dataset but don't have missing revenue in the imdb dataset
(~global_revenue_nans.isin(revenue_nans)).value_counts()

Series([], Name: wikipedia_movie_id, dtype: int64)

1444 scraped movies have revenue informations not present in out original dataset. We can thus complete our revenue data with them.

### Completting missing date data

In [110]:
#imdb_movies.drop(columns=['name'], inplace=True)
#imdb_movies["release_year"] = imdb_movies["release_year"].apply(lambda x: str(x)[:4] if x is not None else x)

#imdb_movies["imdb_release_date"] = pd.to_datetime(imdb_movies["release_year"], errors='coerce', format="%Y")
imdb_movies["imdb_release_date"] = pd.to_datetime(imdb_movies["release_year"], format="%Y")

movies_augmented = pd.merge(movies, imdb_movies, on="wikipedia_movie_id")
movies_augmented.head(3)

ValueError: unconverted data remains: .0

Similarly to the previous part, we see that the scraping was able to retrieve 1711 release years that were missing in our original data. We can similarly complete them.

In [ ]:
missing_dates = movies_augmented[movies_augmented['release_date'].isna()].index.values

In [ ]:
# filling the missing dates with the imdb dates
movies_augmented.loc[missing_dates, "release_date"] = movies_augmented.loc[missing_dates, "imdb_release_date"]

# replace NaT with nans
movies_augmented["release_date"].where(movies_augmented["release_date"].notnull(), None, inplace=True)
movies_augmented.drop(columns=['release_year', 'imdb_release_date'], inplace=True)
movies_augmented.loc[missing_dates, "release_date"].head(3)

We can also create a year feature in our dataset

In [ ]:
movies_augmented.head(3)

In [ ]:
"""extracting the year from the release date feature"""

# extracting the years from the release date feature
movie_with_date = movies_augmented[-movies_augmented["release_date"].isna()].copy(deep=True)
dates = movie_with_date["release_date"]
date_years = dates.astype(str).str.extract(r'(\d{4})')

# adding an index column & renaiming the date_years columns
date_years["index"] = date_years.index
date_years.columns = ["year", "index"]

# adding an index column to the movies_augmented dataframe
movies_augmented["index"] = movies_augmented.index

# merging the movies_augmented and date_years dataframes on the index column
merged = pd.merge(movies_augmented, date_years, on="index", how="left")
merged.drop(columns=["index"], inplace=True)
movies_augmented = merged.copy(deep=True)
movies_augmented.head(3)

In [ ]:
# filling the missing dates with the imdb dates
missing_revenue = movies_augmented[movies_augmented['revenue'].isna()].index.values
movies_augmented.loc[missing_revenue, "revenue"] = movies_augmented.loc[missing_revenue, "global_revenue"]
movies_augmented.loc[missing_revenue, "revenue"].head(3)

### Removing useless data
We shouldn't consider movies with no revenue data, no valid date or no valid ratingwe shouldn't consider movies with no revenue data, no valid date or no valid rating

In [ ]:
#droping missing revenue, date & rating datapoints
revenue_nans_filter         = movies_augmented['revenue'].isna()
release_date_nans_filter    = movies_augmented['release_date'].isna()
rating_nans_filter          = movies_augmented['rating_score'].isna()

movies_augmented = movies_augmented[~(revenue_nans_filter | release_date_nans_filter | rating_nans_filter)].copy(deep=True)

# dropping the global_revenue column
movies_augmented.drop(columns=["global_revenue"], inplace=True)

# checking we removed all the useless datapoints
missing_revenue = movies_augmented["revenue"].isna().sum()
missing_dates = movies_augmented["release_date"].isna().sum()
missing_rating = movies_augmented["rating_score"].isna().sum()

print(f"Missing release date datapoints: {missing_revenue}")
print(f"Missing rating datapoints: {missing_dates}")
print(f"Missing revenue datapoints: {missing_rating}")

In [ ]:
movies_augmented.head(3)

In [ ]:
movies_augmented.shape

We are left with close to 18000 clean datapoints, which is a lot less than the total numbers of movies we had at the beginning. But this is necessary to correctly define the next analysis steps we will conduct.

In [ ]:
# converting years to floats & defining a new dataframe
movies = movies_augmented.copy(deep=True)
movies["year"] = movies["year"].astype(float)

## Taking care of inflation

Comparing movie revenues across different periods of time is not relevant as the value of the USD has varied inbetween. This calls for taking inflation into account and adjusting revenues across all time periods on a single reference date.

In [ ]:
# inflation data from:
# https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/
us_inflation = pd.read_csv(ADDITIONAL_FOLDER + "US_yearly_inflation.csv")
us_yearly_inflation = us_inflation[["Year", "Avg-Avg"]].fillna(0)
us_yearly_inflation.rename(columns={"Year": "year", "Avg-Avg": "avg_avg"}, inplace=True)
us_yearly_inflation.head(3)

In [ ]:
# the 1897 datapoint needs to be removed as we have no inflation data for before 1913
movies = movies[movies["year"] >= 1913]

In [ ]:
# plotting the yearly USD Inflation Rates
plt.plot(us_yearly_inflation['year'], us_yearly_inflation['avg_avg'])
plt.title("Yearly USD Inflation Rates (From Average to Average)")
plt.xlabel("Year")
plt.ylabel("Inflation Rate (%)")
plt.grid(False)
plt.show()

In [ ]:
prices = [1] # we set the initial price as a reference

# Calculate prices based on inflation rates
for index, row in us_yearly_inflation.iterrows():
    if index == 0:
        continue # Skip the first row (no inflation data for the initial price)
    price = prices[-1] * (1 + row['avg_avg'] / 100)
    prices.append(price)

# Add the prices to the DataFrame
us_yearly_inflation['price'] = prices

In [ ]:
reference_price = us_yearly_inflation[us_yearly_inflation["year"]==2012]["price"].values[0]
correction_factor = reference_price / us_yearly_inflation["price"]
us_yearly_inflation["correlation_factor"] = correction_factor
us_yearly_inflation.drop(columns=['price', 'avg_avg'], inplace=True)

merged_all = pd.merge(movies, us_yearly_inflation, on="year", how="left")
movies = merged_all.copy(deep=True)

# inflation adjusted revenue
movies["adjusted_revenue"] = movies["revenue"] * movies["correlation_factor"]
movies.head(3)

### Box office revenue

We need to adjust movie revenues to inflation to make them comparable. As we have inflation data up to 2012, we will place our reference USD value at year 2012.

In [ ]:
# Calculate the max box office revenue for each year
max_revenue_per_year = movies.groupby('year')['revenue'].max().reset_index()
max_revenue_per_year_adjusted = movies.groupby('year')['adjusted_revenue'].max().reset_index()


fig, axes = plt.subplots(1, 2, figsize=(20, 6))
# Create a line plot
axes[0].plot(max_revenue_per_year['year'], max_revenue_per_year['revenue'])
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Max Revenue')
axes[0].set_title('Max Box Office Revenue per Year for non-adjusted inflation')
plt.grid(False)

# Create a line plot
axes[1].plot(max_revenue_per_year_adjusted['year'], max_revenue_per_year_adjusted['adjusted_revenue'])
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Max Revenue')
axes[1].set_title('Max Box Office Revenue per Year for adjusted inflation')
plt.grid(False)
plt.grid(False)

plt.show()

We can plot the max box office revenue for every year. Note that we are interested in the best movies so we don't plot average or median.

In [ ]:
# movie with most revenue
highest_revenue_movie = movies[movies["revenue"]==movies["revenue"].max()]["name"].values[0]

# movie with most revenue
highest_adjusted_revenue_movie = movies[movies["adjusted_revenue"]==movies["adjusted_revenue"].max()]["name"].values[0]

print(f"Highest revenue movie: {highest_revenue_movie}")
print(f"Highest adjusted revenue movie: {highest_adjusted_revenue_movie}")

Looking at the previous graph it seems like Avatar is the best movie! Until we adjust the revenue for inflation and see that it is actually Snow White and the Seven Dwarfs that is the movie that generated the most value.

In [ ]:
movies.head(3)

## Looking for a correlation between the rating and the revenue

In [ ]:
pearson_corr = stats.pearsonr(movies['rating_score'], movies['adjusted_revenue'])

print('Pearson correlation coefficient: ', pearson_corr)
print('As we can see there is a weak correlation between the rating and the revenue')

The null-hypothesis states that there is no correlation between movie revenue and ratings. Based on this Pearson Correlation test, the p-value is smaller than 0.05, so we reject the null-hypothesis. However, the correlation coefficient is equal to 0.14, which is quite small. So we conclude that there is not a significant correlation between movie revenue and ratings.

### Therefore, we can define a success movie score, which combines both the revenue and ratings.

We define the score as a function composed of two components:

- $BORC$    : Box Office Revenue Component
- $RC$      : Rating Component

These components can be computed with the Adjusted Movie Revenue (which we write define as $AMR$) and the movie rating (we will write as $MR$)

We log-transform the data and define $x = \log(AMR)$ and $y = MR$

$
\begin{cases}
    BORC = \frac{x - \min(x)}{\max(x) - \min(x)}\\
    RC = \frac{y - \min(y)}{\max(y) - \min(y)}
\end{cases}
$

As such, we have $BORC, RC \in [0, 1]$

We define a weight $\alpha \in [0, 1]$ and take the convex combinations of $BORC$ and $RC$. This weight controls the importance we give to each of our two components in our metric. Its importance will be determined in the future and we will adjust it accordingly to the study. Multiplying by 100 gives us a final score 

$$Movie Score = 100\left(\alpha BORC + (1 - \alpha) RC \right)$$

In [ ]:
# Calculate the components
log_adj_revenue = np.log(movies['adjusted_revenue'].values)
movie_rating = movies['rating_score'].values

movies.loc[:, 'BORC'] = (log_adj_revenue - np.min(log_adj_revenue)) / (np.max(log_adj_revenue) - np.min(log_adj_revenue))
movies.loc[:, 'RC'] = (movie_rating - np.min(movie_rating)) / (np.max(movie_rating) - np.min(movie_rating))

# Calculate Movie Score
alpha = 1/2
movies.loc[:, 'movie_score'] = 100 * (alpha*movies['BORC'] + (1 - alpha)*movies['RC'])

movies.head(3)

### Now that we have the movie scores we can keep the top T percent and consider these as the Good films that we want to study

In [ ]:
T = 75
num_movies  = movies.shape[0]
movies      = movies.sort_values("movie_score", ascending = False)

df_good_movies  = movies[movies["movie_score"] >= T].copy()
df_bad_movies   = movies[movies["movie_score"] < T].copy()

In [ ]:
df_good_movies.head(3)

In [ ]:
df_bad_movies.head(3)

We now have 2 datasets: of good movies and of bad ones

# Features
## We will find the effect of each feature on the movie’s success score. 

In [ ]:
processed_directory = ADDITIONAL_FOLDER
try:
    os.makedirs(processed_directory)
except:
    pass

In [ ]:
merged = movies.copy(deep=True)

In [ ]:
merged["countries"].iloc[5]

In [ ]:
for key in ast.literal_eval(merged["countries"][5]).keys():
    print(key)

In [ ]:
# country dataframe & dictionary
try:
    movie_countries     = pd.read_csv(processed_directory + "/movie_countries.csv")
    country_dictionary  = pickle.load(open(processed_directory + "/country_dictionary.pickle", "rb"))

except:
    movie_countries = pd.DataFrame(columns=["wikipedia_movie_id", "country_id"])
    country_dictionary = {}
    for i in tqdm(range(merged.shape[0])):
        wikiID = merged["wikipedia_movie_id"][i]
        dico = ast.literal_eval(merged["countries"][i])
        data = [{'wikipedia_movie_id': wikiID, 'country_id': key} for key in dico.keys()]
        dataframe = pd.DataFrame(data)
        movie_countries = pd.concat([movie_countries, dataframe], axis=0)
        country_dictionary.update(dico)

    # save the dataframe to csv
    movie_countries.to_csv(processed_directory + "/movie_countries.csv", index=False)

    # dump the dictionary to pickle
    with open(processed_directory + '/country_dictionary.pickle', 'wb') as handle:
        pickle.dump(country_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# language dataframe & dictionary
try:
    movie_languages     = pd.read_csv(processed_directory + "/movie_languages.csv")
    language_dictionary = pickle.load(open(processed_directory + "/language_dictionary.pickle", "rb"))
except:
    movie_languages = pd.DataFrame(columns=["wikipedia_movie_id", "language_id"])
    language_dictionary = {}
    for i in tqdm(range(merged.shape[0])):
        wikiID = merged["wikipedia_movie_id"][i]
        dico = ast.literal_eval(merged["languages"][i])
        data = [{'wikipedia_movie_id': wikiID, 'language_id': key} for key in dico.keys()]
        dataframe = pd.DataFrame(data)
        movie_languages = pd.concat([movie_languages, dataframe], axis=0)
        language_dictionary.update(dico)

    # save the dataframe to csv
    movie_languages.to_csv(processed_directory + "/movie_languages.csv", index=False)

    # dump the dictionary to pickle
    with open(processed_directory + '/language_dictionary.pickle', 'wb') as handle:
        pickle.dump(language_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
try:
    movie_genres        = pd.read_csv(processed_directory + "/movie_genres.csv")
    genre_dictionary   = pickle.load(open(processed_directory + "/genre_dictionary.pickle", "rb"))
except:
    movie_genres = pd.DataFrame(columns=["wikipedia_movie_id", "genre_id"])
    genre_dictionary = {}
    for i in tqdm(range(merged.shape[0])):
        wiki_id = movies["wikipedia_movie_id"][i]
        dico = ast.literal_eval(merged["genres"][i])
        data = [{'wikipedia_movie_id': wiki_id, 'genre_id': key} for key in dico.keys()]
        dataframe = pd.DataFrame(data)
        movie_genres = pd.concat([movie_genres, dataframe], axis=0)
        genre_dictionary.update(dico)
    
    # save the dataframe to csv
    movie_genres.to_csv(processed_directory + "/movie_genres.csv", index=False)

    # dump the dictionary to pickle
    with open(processed_directory + '/genre_dictionary.pickle', 'wb') as handle:
        pickle.dump(genre_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

We can now start by properly merging in the two new dataframes we created:

In [ ]:
merged_languages    = pd.merge(merged, movie_languages, on="wikipedia_movie_id", how="inner")
merged_countries    = pd.merge(merged_languages, movie_countries, on="wikipedia_movie_id", how="inner")
merged_all          = pd.merge(merged_countries, movie_genres, on="wikipedia_movie_id", how="inner")
merged_all.drop(columns=["languages", "countries", "genres"], inplace=True)
merged_all.head(3)

We can also replace the language and country ids by their corresponding values

In [ ]:
merged_all["language_id"].replace(language_dictionary, inplace=True)
merged_all["country_id"].replace(country_dictionary, inplace=True)
merged_all["genre_id"].replace(genre_dictionary, inplace=True)

#rename language_id as language, country_id as country and genre_id as genre
merged_all.rename(columns={"language_id": "language", "country_id": "country", "genre_id": "genre"}, inplace=True)
merged_all.head(3)

In [ ]:
inflation_group = merged_all.groupby(["country"]).agg({"adjusted_revenue": "sum"})
inflation_group.sort_values(by="adjusted_revenue", ascending=False).head(20).plot(kind="bar", title="Total revenue per country adjusted for inflation", xlabel="Country", ylabel="Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012")

In [ ]:
# Top 20 countries that have generated the most total revenue
top_20_countries = inflation_group.sort_values(by="adjusted_revenue", ascending=False).head(20).index.values

In [ ]:
# plotting the average revenue per language adjusted for inflation
inflation_language_group = merged_all.groupby(["language"]).agg({"adjusted_revenue": "sum"})
inflation_language_group.sort_values(by="adjusted_revenue", ascending=False).head(20).plot(kind="bar", title="Total revenue per language adjusted for inflation", xlabel="Language", ylabel="Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012")

Following the evolution over the years of the 20 countries that have generated the most total revenue

In [ ]:
top_20_merged   = merged_all[merged_all["country"].isin(top_20_countries)]
top_20_merged.dropna(subset=["adjusted_revenue"], inplace=True)
top_20_grouped  = top_20_merged.groupby(["year", "country"])
top_20_sum      = top_20_grouped.agg({"adjusted_revenue": "sum"})

In [ ]:
ax = top_20_sum.unstack().plot(kind='bar', stacked=True, title="Total revenue per country adjusted for inflation", xlabel="year", ylabel="Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012", figsize=(20,10))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

We clearly see that globally, the movie industry has been making more money over the years. Though we see a drop during the years that follow 2008 which might be a consequence of the subprime's crisis.

The US dominance remains present even when looking at year-by-year movies revenue adjusted for inflation. The US has been the most productive country in terms of movies production and revenue generation for the past 100 years.

# Revenue analysis over the length of a movie

Aboout 35 movies have a runtime that is longer that 500 minutes, which does not make sense.

In [ ]:
merged_all[merged_all["runtime"]>=500]["runtime"].describe()

We can remove these outliers and perform a `pearson correlation` test between the `movie_runtime` and its `adjusted_revenue`

In [ ]:
merged_with_inflation_no_outlier    = merged_all[merged_all["runtime"]<500]
non_nan_merged_with_inflation       = merged_with_inflation_no_outlier[~merged_with_inflation_no_outlier["adjusted_revenue"].isna()]
non_nan_merged_with_inflation       = non_nan_merged_with_inflation[~non_nan_merged_with_inflation["runtime"].isna()]
stats.pearsonr(non_nan_merged_with_inflation["runtime"], non_nan_merged_with_inflation["adjusted_revenue"])

The p_value is low enough, we can trust the value of Pearson's coefficient `r=0.21` which indicates a weak positive correlation between `runtime` of a movie and its `adjusted_revenue`

In [ ]:
# plotting revenue vs runtime with a linear regression
sns.lmplot(x="runtime", y="adjusted_revenue", data=non_nan_merged_with_inflation)
plt.xlabel("Runtime in minutes")
plt.ylabel("Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012")
plt.title("Revenue vs Runtime")

In [ ]:
non_nan_merged_with_inflation["runtime"].describe()

# Augmented data analysis

Plotting the 20 producers that have generated the most revenue over their carreer 

In [ ]:
scraped_grouped = merged_all.groupby(["producer"]).agg({"adjusted_revenue": "sum"})
scraped_grouped.sort_values(by="adjusted_revenue", ascending=False).head(20).plot(kind="bar", title="Total revenue per producer adjusted for inflation\n for top 20 generating producers", xlabel="Producer", ylabel="Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012")

`Adjusted Revenue` and `Rating` correlation analysis

In [ ]:
# plot revenue adjusted against rating score
merged_all.plot(kind="scatter", x="rating_score", y="adjusted_revenue", title="Revenue vs Rating", xlabel="Rating", ylabel="Equivalent revenue in USD adjusted \n for inflation with 1$ reference of 2012")

In [ ]:
non_nan_scraped_merge = merged_all[~merged_all["rating_score"].isna()]
non_nan_scraped_merge = non_nan_scraped_merge[~non_nan_scraped_merge["adjusted_revenue"].isna()]
stats.pearsonr(non_nan_scraped_merge["rating_score"], non_nan_scraped_merge["adjusted_revenue"])

The p_value is low enough, we can trust the value of Pearson's coefficient `r=0.14` which indicates a weak positive correlation between `rating score` of a movie and its `adjusted revenue`

## 4) Movie budget

## 5) Genres

In [ ]:
movies_genres = movies.copy(deep=True)
movies_genres.head()

In [ ]:
def extract_json_values(df, column):
    """
    This function takes a dataframe and a column name containing JSON strings.
    It parses the JSON and preserves the content/values as a list of lists.
    """
    # Extract the values from each JSON object and append to a list
    values_list = []
    for json_str in df[column]:
        try:
            # Parse the JSON data
            json_data = json.loads(json_str)
            # Extract values and append to the list
            values_list.append(list(json_data.values()))
        except json.JSONDecodeError:
            # If JSON is not valid, append a None or handle it as needed
            values_list.append(None)
            
    return values_list

In [ ]:
movies_genres['genres'] = extract_json_values(movies_genres, "genres")
movies_genres.head(10)

Let’s see how many movies there are per gender.

In [ ]:
# Assuming your DataFrame is named 'movies'
# Replace 'YourColumnName' with the actual column name in your DataFrame
genres = movies_genres['genres']

# Create an empty dictionary to store genre counts
genres_counts = {}

# Iterate through each list of genres in the 'genres' column
for genres_list in genres:
    # Update the counts in the dictionary
    for genre in genres_list:
        genres_counts[genre] = genres_counts.get(genre, 0) + 1

# Convert the dictionary to a DataFrame for better visualization
genres_counts_df = pd.DataFrame(list(genres_counts.items()), columns=['Genre', 'Count'])

# Sort the DataFrame by the count in descending order
genres_counts_df = genres_counts_df.sort_values(by='Count', ascending=False)

# Display the result
genres_counts_df.head(362)

As we can see above, there are many genres who are very similar (technically the same but differ in terms of expression): (Comedy = Comedy film = Comedy, …), and some of them are outliers (very few movies per genre). Moreover, the 16 most popular genres are: 
`Action`, `Adventure`, `Animation`, `Comedy`, `Crime`, `Drama`, `Family`, `Fantasy`, `Period Piece`, `Horror`, `Musical`, `Romance`, `Science Fiction`, `Sport`, `Thriller and War film`. 
Therefore, we regroup all the movies into the 16 genres above (the movies who have the same genre or a very similar one). 

In [ ]:
# Assuming your DataFrame with genre counts is named 'genre_counts_df'
# Replace 'YourColumnName' with the actual column name in your DataFrame
genres_column = genres_counts_df['Genre']

# Define the genres of interest
primary_genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 
                   'Fantasy', 'Period piece', 'Horror', 'Musical', 'Romance', 'Science Fiction', 
                   'Sport', 'Thriller', 'War film']

# Create a dictionary to store the grouped genres and counts
grouped_genre_data = {'Primary Genre': [], 'Grouped Genres': [], 'Count': []}

# Group genres based on keywords
for primary_genre in primary_genres:
    # Find genres that contain the keyword
    grouped_genres = [genre for genre in genres_column if primary_genre.lower() in genre.lower()]
    
    # Sum the counts for the grouped genres
    grouped_count = genres_counts_df[genres_counts_df['Genre'].isin(grouped_genres)]['Count'].sum()
    
    # Add data to the dictionary
    grouped_genre_data['Primary Genre'].append(primary_genre)
    grouped_genre_data['Grouped Genres'].append(', '.join(grouped_genres))
    grouped_genre_data['Count'].append(grouped_count)

# Create a new DataFrame from the grouped data dictionary
grouped_genre_counts_df = pd.DataFrame(grouped_genre_data)

# Reorder the DataFrame
grouped_genre_counts_df = grouped_genre_counts_df.sort_values(by='Count', ascending=False)

# Display the result
grouped_genre_counts_df.head(18)

We will create a new column called `Generic Genre`, which is the same as the genres column but keeping only the 16 most popular genres, and removing the rest. 

In [ ]:
movies_genres['Generic Genre'] = np.nan
movies_genres.head(10)

In [ ]:
# Assuming 'movies' is your DataFrame
# If not, you need to define or read it before this point

# List of genres to create tables for
genres_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family',
               'Fantasy', 'Period piece', 'Horror', 'Musical', 'Romance', 'Science Fiction',
               'Sport', 'Thriller', 'War film']

# Define a function to get the list of genres for a movie
def get_genre_list(movie_genres):
    return [genre for genre in genres_list if any(genre.lower() in element.lower().strip() for element in movie_genres)]

# Apply the function to create the 'Generic Genre' column
movies_genres['Generic Genre'] = movies_genres['genres'].apply(get_genre_list)

movies_genres.head(10)

We will now create a dictionary consisting of 16 different dataframes (1 for each genre). Each one of these dataframes corresponds to all the movie of the specific genre. 

In [ ]:
genres_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family',
               'Fantasy', 'Period piece', 'Horror', 'Musical', 'Romance', 'Science Fiction',
               'Sport', 'Thriller', 'War film']
#for genre in genres_list: 
#    genre_movies = movies[movies['Generic Genre'].apply(lambda x: genre in x if x is not None else False)]
#   genre_movies.head(10)

genre_dataframes = {}

for genre in genres_list:
    genre_movies = movies_genres[movies_genres['Generic Genre'].apply(lambda x: genre in x if x is not None else False)]
    genre_dataframes[genre] = genre_movies

# Access specific genre DataFrames using genre names
action_movies = genre_dataframes['Action']
adventure_movies = genre_dataframes['Adventure']
animation_movies = genre_dataframes['Animation']
comedy_movies = genre_dataframes['Comedy']
crime_movies = genre_dataframes['Crime']
drama_movies = genre_dataframes['Drama']
family_movies = genre_dataframes['Family']
fantasy_movies = genre_dataframes['Fantasy']
period_piece_movies = genre_dataframes['Period piece']
horror_movies = genre_dataframes['Horror']
musical_movies = genre_dataframes['Musical']
romance_movies = genre_dataframes['Romance']
scifi_movies = genre_dataframes['Science Fiction']
sport_movies = genre_dataframes['Sport']
thriller_movies = genre_dataframes['Thriller']
war_movies = genre_dataframes['War film']

Let’s now do a distribution of the movie score per genre.

In [ ]:
# Assuming 'genre_dataframes' is the dictionary of genre-specific DataFrames

# Create a 4x4 subplot grid for all genres except 'Action'
fig, axes = plt.subplots(4, 4, figsize=(15, 15), sharex=True, sharey=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Iterate through the genres (excluding 'Action') and corresponding DataFrames
for i, (genre, genre_df) in enumerate(genre_dataframes.items()):
    if genre != 'Action':
        plt.xscale("log")
        # Plot the distribution of movie revenue for each genre
        sns.histplot(ax=axes[i], data=genre_df, x='movie_score', kde=True, bins=20, color='C{}'.format(i))
        axes[i].set_title(genre)
        axes[i].set_xlabel('Movie Score')
        axes[i].set_ylabel('Count')

# Plot 'Action' genre in the middle of the 5th row, 16th subplot
action_df = genre_dataframes['Action']
# Set x-axis to log scale for the 'Action' subplot
axes[0].set_xscale('log')
sns.histplot(ax=axes[0], data=action_df, x='movie_score', kde=True, bins=20)
axes[0].set_title('Action')
axes[0].set_xlabel('Movie Score')
axes[0].set_ylabel('Count')

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# Revenue statistics per genre
# Create empty dictionaries to store statistics for each genre
genre_stats = {'Mean': {}, 'Median': {}, 'Maximum': {}, 'Minimum': {}}

# Iterate through the genres and corresponding DataFrames
for genre, genre_df in genre_dataframes.items():
    mean_score = genre_df['movie_score'].mean()
    median_score = genre_df['movie_score'].median()
    max_score = genre_df['movie_score'].max()
    min_score = genre_df['movie_score'].min()

    # Store the results in the dictionaries
    genre_stats['Mean'][genre] = mean_score
    genre_stats['Median'][genre] = median_score
    genre_stats['Maximum'][genre] = max_score
    genre_stats['Minimum'][genre] = min_score

# For easy displays, convert the dictionaries to Pandas DataFrames 
mean_df = pd.DataFrame(genre_stats['Mean'], index=['Mean Movie Score'])
median_df = pd.DataFrame(genre_stats['Median'], index=['Median Movie Score'])
max_df = pd.DataFrame(genre_stats['Maximum'], index=['Maximum Movie Score'])
min_df = pd.DataFrame(genre_stats['Minimum'], index=['Minimum Movie Score'])

# Concatenate the DataFrames along the columns to create a general DataFrame
general_stats_df = pd.concat([mean_df, median_df, max_df, min_df])

# Display the summary DataFrame
general_stats_df.head(10)

In [ ]:
# Identify which genre has the highest and lowest values for each statistic
# Mean 
highest_mean_genre = general_stats_df.loc['Mean Movie Score'].idxmax()
lowest_mean_genre = general_stats_df.loc['Mean Movie Score'].idxmin()

# Display the results
print(f"Highest Mean Movie Score Genre: {highest_mean_genre}")
print(f"Lowest Mean Movie Score Genre: {lowest_mean_genre}")

In [ ]:
# Median
highest_median_genre = general_stats_df.loc['Median Movie Score'].idxmax()
lowest_median_genre = general_stats_df.loc['Median Movie Score'].idxmin()

print(f"Highest Median Movie Score Genre: {highest_median_genre}")
print(f"Lowest Median Movie Score Genre: {lowest_median_genre}")

In [ ]:
# Max
highest_max_genre = general_stats_df.loc['Maximum Movie Score'].idxmax()
lowest_max_genre = general_stats_df.loc['Maximum Movie Score'].idxmin()

print(f"Highest Maximum Movie Score Genre: {highest_max_genre}")
print(f"Lowest Maximum Movie Score Genre: {lowest_max_genre}")

In [ ]:
# Min
highest_min_genre = general_stats_df.loc['Minimum Movie Score'].idxmax()
lowest_min_genre = general_stats_df.loc['Minimum Movie Score'].idxmin()

print(f"Highest Minimum Movie Score Genre: {highest_min_genre}")
print(f"Lowest Minimum Movie Score Genre: {lowest_min_genre}")

In [ ]:
# Create a list to store DataFrames for box plots
boxplot_genre_dfs = []

# Iterate through the genres and corresponding DataFrames
for genre, genre_df in genre_dataframes.items():
    # Use .copy() to create a copy and add a 'Genre' column
    genre_df_copy = genre_df.copy()
    genre_df_copy['Genre'] = genre
    boxplot_genre_dfs.append(genre_df_copy[['movie_score', 'Genre']])


# Concatenate the DataFrames along the rows for box plot
boxplot_concat = pd.concat(boxplot_genre_dfs)

# Create a box plot using Seaborn
plt.figure(figsize=(15, 8))
sns.boxplot(x='Genre', y='movie_score', data=boxplot_concat, showfliers=False)
plt.title('Box Plot of Movie Score for Each Genre')
plt.xlabel('Genre')
plt.ylabel('Movie Score')
# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()

We will now find some statistics for each genre

In [ ]:
genre_stats = {}

for genre, genre_df in genre_dataframes.items():
    genre_stats[genre] = genre_df['movie_score'].describe()

print(genre_stats)

## 6) Inclusivity

Let’s begin by retrieving actors’ information’s.

In [ ]:
col_names = ['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 'date_of_birth', 'gender', 'height', 'ethnicity_id', 'name', 'age_at_release', 'freebase_character_map_id', 'freebase_character_id', 'freebase_actor_id']

characters = pd.read_csv(DATA_FOLDER + 'character.metadata.tsv', sep='\t', header=None,  names=col_names)
characters.head()

In [ ]:
actors = characters[['date_of_birth',
    'gender',
    'height',
    'ethnicity_id',
    'name',
    'freebase_actor_id']]
non_duped_actors = actors.drop_duplicates()
non_duped_actors.head()

In [ ]:
#Remove actors with non given name
named_actors = non_duped_actors[~non_duped_actors['name'].isna() ]
named_actors.head()

To look at the data, we now find out the proportion of ethnicity ids presence between actors accros all films:

In [ ]:
actors_in_movie_eth = characters[['name','freebase_actor_id','wikipedia_movie_id','ethnicity_id']].drop_duplicates()
actors_in_movie_eth = actors_in_movie_eth[~actors_in_movie_eth['name'].isna()].drop(columns=['name'])
actors_in_movie_eth_counts = actors_in_movie_eth.groupby('freebase_actor_id').count().rename(columns = {'wikipedia_movie_id': 'Movie count', 'ethnicity_id': 'Number of Ethnicity tags'})
actors_in_movie_eth_counts = actors_in_movie_eth_counts[actors_in_movie_eth_counts['Movie count'] >= 1]
print(actors_in_movie_eth_counts[actors_in_movie_eth_counts['Number of Ethnicity tags']==0].sum())




In [ ]:
actors_with_eth_tags_count = actors_in_movie_eth_counts[actors_in_movie_eth_counts['Number of Ethnicity tags']!= 0]
print(actors_with_eth_tags_count.sum())


Below are all the actors with tags :

In [ ]:
actor_eth = characters[['name','freebase_actor_id','ethnicity_id']].drop_duplicates()
actor_eth = actor_eth[~actor_eth['name'].isna()].drop(columns=['name'])
actor_eth
actor_eth[~actor_eth['ethnicity_id'].isna()]

The below graphs shows the distribution of ethnicity tags and actors existance in film:

In [ ]:
actors_in_movie_eth = characters[['name','freebase_actor_id','wikipedia_movie_id','ethnicity_id']].drop_duplicates()
actors_in_movie_eth = actors_in_movie_eth[~actors_in_movie_eth['name'].isna()].drop(columns=['name'])
actors_in_movie_eth.groupby('wikipedia_movie_id').count().describe()
actors_in_movie_eth.groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,5])


Below is the movie's distinctive ethnicity tag repartitions considering NaN values:    
Note that per movie we consider each actor once.

In [ ]:
eth_in_movie_eth = characters[['name','freebase_actor_id','wikipedia_movie_id','ethnicity_id']].drop_duplicates()
eth_in_movie_eth = eth_in_movie_eth[~eth_in_movie_eth['name'].isna()].drop(columns=['name', 'freebase_actor_id'])
eth_in_movie_eth = eth_in_movie_eth.drop_duplicates()
eth_in_movie_eth.groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,5])

We start our study on ethnicity by selecting movies that had sufficient information about the ethnicity in their cast. This leads us to define a certain threshold on the number of actors' ethnicity tags per film, and only consider the movies that have a number of tags larger than this threshold.   
On these movies, we may now compute the number of different tags of each movie and plotting histograms of the new data.
The movies which have a number of different tags larger than our defined threshold are considered as inclusive movies, while the others are considered as non-inclusive.    
The value of the threshold may be adapted in the future to see how it affects the results.

In [ ]:
thresh = 3

In [ ]:
eth_in_movie_eth = characters[['name','freebase_actor_id','wikipedia_movie_id','ethnicity_id']].drop_duplicates()
eth_in_movie_eth = eth_in_movie_eth[~eth_in_movie_eth['name'].isna()].drop(columns=['name', 'freebase_actor_id'])

eth_in_movie_eth = eth_in_movie_eth[~eth_in_movie_eth['ethnicity_id'].isna()]
count_by_movie_id = eth_in_movie_eth.groupby('wikipedia_movie_id').count()

#Film with infos on 3 ethnicities
filtered_data = eth_in_movie_eth[eth_in_movie_eth['wikipedia_movie_id'].isin(count_by_movie_id[count_by_movie_id >= thresh].dropna().index)]
#Films with possible duplicates of these 3 ethnicities ie the movies for which we have sufficient data to determine whether they are inclusive or not.
filtered_data.groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,thresh+3])

#Films with at least distinct 3 ethnicities among these movies for which we have sufficient data
filtered_data.drop_duplicates().groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,thresh+3])

But before starting analysing the data, we need to match the given ethnicity ids with their respective values from freebase and wikidata      
Below is the scraping :

In [ ]:
eth_indexes = actors['ethnicity_id'].value_counts().index
eth_indexes

In [ ]:
actors['ethnicity_id'].isna().sum()

In [ ]:
actors[~actors['ethnicity_id'].isna()]

In [ ]:
import requests

def get_wikidata_id(freebase_id):
    """
    Function fetching from the wikidata_id of the given freebase_id
    """
    # Wikidata SPARQL endpoint
    endpoint_url = "https://query.wikidata.org/sparql"
    
    # SPARQL query to get the Wikidata ID using the Freebase ID
    query = f'''
    SELECT ?item WHERE {{
        ?item wdt:P646 "{freebase_id}".
    }}
    '''
    
    # Send the request to the endpoint
    response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
    while response.status_code == 429:
        time.sleep(2.5)
        print('sleeping')
        response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
        print('Woke up !')
    # Parse the response
    data = response.json()
    
    # Extract the Wikidata ID from the response
    results = data.get("results", {}).get("bindings", [])
    if results:
        wikidata_id = results[0].get("item", {}).get("value", "")
        # Extract the ID from the full URI
        wikidata_id = wikidata_id.split('/')[-1]
        return wikidata_id
    else:
        return None


In [ ]:
def get_ethnicity_info(wikidata_id):
    """
    Function fetching the ethnicity information using the wikidata_id
    """
    endpoint_url = "https://query.wikidata.org/sparql"
    query = f'''
    SELECT ?property ?propertyLabel ?value ?valueLabel WHERE {{
        wd:{wikidata_id} ?p ?statement .
        ?statement ?ps ?value .
        
        ?property wikibase:claim ?p.
        ?property wikibase:statementProperty ?ps.
        
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    ORDER BY ?property ?value
    '''
    
    # Send the request to the endpoint
    response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
    while response.status_code == 429:
        time.sleep(2.5)
        print('sleeping')
        response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
        print('Woke up !')
        
    # Parse the response
    data = response.json()
    # Process the data to extract labels and values
    properties = {}
    for item in data.get("results", {}).get("bindings", []):
        prop = item.get("propertyLabel", {}).get("value", "")
        val = item.get("valueLabel", {}).get("value", "")
        if prop and val:
            if prop in properties:
                properties[prop].append(val)
            else:
                properties[prop] = [val]
        
    return properties


In [ ]:
# Mock example :
freebase_id =  '/m/043_z22'
wikidata_id = get_wikidata_id(freebase_id)
print(wikidata_id)
if wikidata_id:
    ethnicity_info = get_ethnicity_info(wikidata_id)
    print(ethnicity_info['instance of'])
else:
    print("Wikidata ID not found for the given Freebase ID.")


In [ ]:
def scrape_ethnicity_infos(eth_indexes):
    """
    Function scraping for a list of ethnicity values using a list of ethnicity freebase id
    """
    free_ids = []
    wiki_ids = []
    eth_infos = []
    for idx, act_eth in enumerate(eth_indexes):
            if idx %10 == 0:
                print("Random sleep", idx)
                time.sleep(2.5)
                print('Woke up !')
                
            freebase_id =  act_eth
            wikidata_id = get_wikidata_id(freebase_id)
            
            free_ids.append(freebase_id)
            wiki_ids.append(wikidata_id)
            if wikidata_id:
                ethnicity_info = get_ethnicity_info(wikidata_id)
                eth_infos.append(ethnicity_info)
            else:
                eth_infos.append(None)
                print("Wikidata ID not found for the given Freebase ID.")
                
    return eth_infos, wiki_ids, free_ids

In [ ]:
def retrieve_ethnicity_infos(eth_indexes=None):
    """
    Function used to retrieve the ethnicity values either from saved data or from scraping
    """
    try:
        with open(ADDITIONAL_FOLDER + 'eth_infos.pkl', 'rb') as file:
            eth_infos = pkl.load(file)
        with open(ADDITIONAL_FOLDER + 'wiki_ids.pkl', 'rb') as file:
            wiki_ids = pkl.load(file)
        with open(ADDITIONAL_FOLDER + 'free_ids.pkl', 'rb') as file:
            free_ids = pkl.load(file)
        return eth_infos, wiki_ids, free_ids
    
    except:
        eth_infos, wiki_ids, free_ids = scrape_ethnicity_infos(eth_indexes)
        with open(ADDITIONAL_FOLDER + 'eth_infos.pkl', 'wb') as file:
            pkl.dump(eth_infos, file)      
        with open(ADDITIONAL_FOLDER + 'wiki_ids.pkl', 'wb') as file:
            pkl.dump(wiki_ids, file)   
        with open(ADDITIONAL_FOLDER + 'free_ids.pkl', 'wb') as file:
            pkl.dump(free_ids, file)
        return eth_infos, wiki_ids, free_ids      

In [ ]:
eth_infos, wiki_ids, free_ids = retrieve_ethnicity_infos(eth_indexes=eth_indexes)
len(eth_infos)

In [ ]:
data_dict = {
    'wiki_ids': wiki_ids,
    'eth_infos': eth_infos,
    'category' : np.nan
}

# Create a ethnicity information DataFrame with free_ids as the index
ethnicity_map = pd.DataFrame(data_dict, index=free_ids)
ethnicity_map['category'] = ethnicity_map['category'].astype('object')
ethnicity_map

In [ ]:
ethnicity_map = ethnicity_map.replace('', np.nan)
ethnicity_map = ethnicity_map[ (~ethnicity_map['eth_infos'].isna() )]

The best way to understand the data is to look at it.
In the following subsection we are going to retrieve little by little information from the quite dirty obtained data :

In [ ]:
ethnicity_map['eth_infos'].iloc[1]

In [ ]:
def remove_stopwords_and_specific_words(text, specific_words):
    """
    Function used to preprocess and clean the ethnicities values obtained by removing stop words and demanded words.
    """
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in specific_words]
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

In [ ]:
class EthnicityColumnProcessing(ABC):
    """
    Super class used to define a generic cleaning policy of the ethnicity values.
    """
    @staticmethod
    def find_category_from_dict(dico):
        """
        Function which return a single cleaned and demanded from a dict.
        """
        raise NotImplementedError("Subclasses must implement this method")
        
    @staticmethod
    def process_data_list(data_list):
        """
        Function used to clean the hole data obtained from the ethnicity dict.
        """
        if data_list is None or data_list is np.nan:
            return np.nan
            
        new_data_list = []  
        for item in data_list:
            processed_content = item.replace('-', ' ').lower()
            processed_content = remove_stopwords_and_specific_words(processed_content, ['people', 'peoples', 'descent'])
            new_data_list.append(processed_content)
        return new_data_list
        
    @classmethod
    def process_dataframe(cls, df):
        """
        Function used to obtain and update the ethnicity category of a dataframe with new informations.
        """
        for index, row in df.iterrows():
            if isinstance(row['category'], list):
                if pd.isna(row['category']).all():
                    df.at[index, 'category'] = cls.process_data_list(cls.find_category_from_dict(row['eth_infos']))
            elif pd.isna(row['category']):
                df.at[index, 'category'] = cls.process_data_list(cls.find_category_from_dict(row['eth_infos']))
        

We incrementally add values from the ethnicity infos `eth_info` to the ethnicity category of our dataframe `ethnicity_map`

In [ ]:
class MainCategoryProcessing(EthnicityColumnProcessing):
    @staticmethod
    def find_category_from_dict(dico):
        category = dico.get("topic's main category", np.nan)   
        return category
        
    @staticmethod
    def process_data_list(data_list):
        if data_list is None or data_list is np.nan:
            return np.nan
            
        new_data = []  
        for item in data_list:
            if item.startswith("Category:"):
                content = item[len("Category:"):]
    
                processed_content = content.replace('-', ' ').lower()
                processed_content = remove_stopwords_and_specific_words(processed_content, ['people', 'peoples', 'descent'])
                new_data.append(processed_content)
            else:
                continue
    
        return new_data

In [ ]:
MainCategoryProcessing().process_dataframe(ethnicity_map)
ethnicity_map

In [ ]:
ethnicity_map['eth_infos'].loc['/m/03x_fq7']

In [ ]:
class CommonCategoryProcessing(EthnicityColumnProcessing):
    @staticmethod
    def find_category_from_dict(dico):
        category = dico.get('Commons category', np.nan)   
        return category

CommonCategoryProcessing().process_dataframe(ethnicity_map)
display(ethnicity_map)
ethnicity_map.isna().sum()

In [ ]:
class EncyclopedyCategoryProcessing(EthnicityColumnProcessing):
    @staticmethod
    def find_category_from_dict(dico):
        if isinstance(dico, dict):
            eth = dico.get('Encyclopædia Britannica Online ID', np.nan)
            if eth is not np.nan:
                eth = eth[0].removeprefix("topic/")
                    
            return eth
        else: 
            return np.nan
        
EncyclopedyCategoryProcessing().process_dataframe(ethnicity_map)
ethnicity_map.isna().sum()

In [ ]:
ethnicity_map.loc['/m/06mvq'].eth_infos

In [ ]:
class QuoraCategoryProcessing(EthnicityColumnProcessing):
    @staticmethod
    def find_category_from_dict(dico):
        category = dico.get('Quora topic ID', np.nan)   
        return category
        
QuoraCategoryProcessing().process_dataframe(ethnicity_map)
ethnicity_map.isna().sum()

In [ ]:
ethnicity_map[(ethnicity_map['category'].isna())]

In [ ]:
ethnicity_map.loc['/m/0g96wd'].eth_infos

In [ ]:
ethnicity_map.loc['/m/02rm7_9'].eth_infos

In [ ]:
class CountryCategoryProcessing(EthnicityColumnProcessing):
    @staticmethod
    def find_category_from_dict(dico):
        category = dico.get('country', np.nan)   
        return category
        
CountryCategoryProcessing().process_dataframe(ethnicity_map)
ethnicity_map.isna().sum()

In [ ]:
ethnicity_map.loc['/m/01hm_'].eth_infos

In [ ]:
ethnicity_map.loc['/m/047q05d'].eth_infos

In [ ]:
ethnicity_map.loc['/m/03b_13l'].eth_infos


In [ ]:
ethnicity_map.loc['/m/0960kn'].eth_infos

We see now that most of the useful obtained values are in our dataframe.   
we therefore stop the incremental dataframe update here.   
We find now all actors with existing mapping:

In [ ]:
# All actors with existing mapping
useful_eth_tags = ethnicity_map[~ethnicity_map['category'].isna()]
useful_eth_tags

actors_with_eth_tags = characters[['name','freebase_actor_id','wikipedia_movie_id','ethnicity_id']].drop_duplicates()
actors_with_eth_tags = actors_with_eth_tags[~actors_with_eth_tags['name'].isna()].drop(columns=['name'])

actors_with_eth_tags = actors_with_eth_tags[~actors_with_eth_tags['ethnicity_id'].isna()]
actors_with_eth_categories = actors_with_eth_tags[actors_with_eth_tags['ethnicity_id'].isin(useful_eth_tags.index)]
actors_with_eth_categories

We now recompute the last cell to check if any drastic changes in the data occured 

In [ ]:
eth_in_movie_eth = actors_with_eth_categories.drop(columns=['freebase_actor_id'])

eth_in_movie_eth = eth_in_movie_eth[~eth_in_movie_eth['ethnicity_id'].isna()]
count_by_movie_id = eth_in_movie_eth.groupby('wikipedia_movie_id').count()

#Film with infos on 3 ethnicities
filtered_data = eth_in_movie_eth[eth_in_movie_eth['wikipedia_movie_id'].isin(count_by_movie_id[count_by_movie_id >= thresh].dropna().index)]

#Films with possible duplicates of these 3 ethnicities ie the movies for which we have sufficient data to determine whether they are inclusive or not.
filtered_data.groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,thresh+3])

#Films with at least distinct 3 ethnicities among these movies for which we have sufficient data
filtered_data.drop_duplicates().groupby('wikipedia_movie_id').count().hist(bins=20, range = [0,thresh+3])



We saw that no drastic change occured except for some loss of movies.

Now let's start our little analysis by first redefining the choosen threshold for separating non inclusive and inclusive films :

In [ ]:
threshold = 3

In [ ]:
film_ethnicity_difference_repartition = filtered_data.drop_duplicates().groupby('wikipedia_movie_id').count()
low_inclusivity_films_ids = film_ethnicity_difference_repartition[film_ethnicity_difference_repartition < threshold].dropna().index
high_inclusivity_films_ids = film_ethnicity_difference_repartition[film_ethnicity_difference_repartition >= threshold].dropna().index

In [ ]:
movies_inclusivity = movies.copy(deep=True)

In [ ]:
low_inclusivity_movies = movies_inclusivity[movies_inclusivity['wikipedia_movie_id'].isin(low_inclusivity_films_ids)]
high_inclusivity_movies = movies_inclusivity[movies_inclusivity['wikipedia_movie_id'].isin(high_inclusivity_films_ids)]
display(low_inclusivity_movies)
display(high_inclusivity_movies)

In [ ]:
inclusive_good_movies = df_good_movies[df_good_movies['wikipedia_movie_id'].isin(high_inclusivity_films_ids)].copy(deep=True)
inclusive_bad_movies = df_bad_movies[df_bad_movies['wikipedia_movie_id'].isin(high_inclusivity_films_ids)].copy(deep=True)
exclusive_good_movies = df_good_movies[df_good_movies['wikipedia_movie_id'].isin(low_inclusivity_films_ids)].copy(deep=True)
exclusive_bad_movies = df_bad_movies[df_bad_movies['wikipedia_movie_id'].isin(low_inclusivity_films_ids)].copy(deep=True)

inc_good_len = len(inclusive_good_movies)
inc_bad_len = len(inclusive_bad_movies)
ex_good_len = len(exclusive_good_movies)
ex_bad_len = len(exclusive_bad_movies)


In [ ]:
print(f"Number of good inclusive movies {inc_good_len}")
print(f"Number of good exclusive movies {ex_good_len} ")
print()
print(f"Number of bad inclusive movies {inc_bad_len}")
print(f"Number of bad exclusive movies {ex_bad_len}")

With a naive and somewhat incorrect analysis, we see that inclusive movies tend to be worse than their counter parts, but somehow more rewarding 
when successful. 
A deeper probabilistic analysis is therefore needed here to know if there is a significant difference between inclusive and exclusive movies success 

As a first step in this direction, lets compute the proportion of inclusive and exclusive movies that are good.

In [ ]:
print(f"Proportion of inclusive movies that are good movies {inc_good_len / (inc_good_len + inc_bad_len)}")
print(f"Proportion of exclusive movies that are good movies {ex_good_len / (ex_good_len + ex_bad_len)}")

In [ ]:
exclusive_mean = low_inclusivity_movies["movie_score"].mean()
inclusive_mean = high_inclusivity_movies["movie_score"].mean()

ex_std = low_inclusivity_movies["movie_score"].std()
in_std = high_inclusivity_movies["movie_score"].std()

e_b = exclusive_mean - 1.96 * ex_std / np.sqrt(low_inclusivity_movies['movie_score'].shape[0])
e_u = exclusive_mean + 1.96 * ex_std / np.sqrt(low_inclusivity_movies['movie_score'].shape[0])

i_b = inclusive_mean - 1.96 * in_std / np.sqrt(high_inclusivity_movies['movie_score'].shape[0])
i_u = inclusive_mean + 1.96 * in_std / np.sqrt(high_inclusivity_movies['movie_score'].shape[0])

print(f"Exclusive movies' mean movie score 95% confidence interval: [{e_b:.4f}, {e_u:.4f}]")
print(f"Inclusive movies' mean movie score 95% confidence interval: [{i_b:.4f}, {i_u:.4f}]")

In [ ]:
#confid = df[~df["salary"].isna()][["salary", "throws"]]
inc = pd.DataFrame()
exl = pd.DataFrame()

inc['movie_score'] = high_inclusivity_movies["movie_score"]
exl['movie_score'] = low_inclusivity_movies["movie_score"]

inc['inclusivity'] = "Highly Inclusive"
exl['inclusivity'] = "Lowly Inclusive"

confid = pd.concat([inc, exl])
#display(confid)
sns.pointplot(x="inclusivity",
              y="movie_score",
              data=confid,
              errorbar=('ci', 95),
              n_boot=10000,
              seed=1,
              )
plt.title('Point plot showing confidence intervals of movie scores in function of inclusivity of the film')
plt.grid(False)
plt.show()


This simplistic analysis therefore shows that inclusive films somewhat performs better than non-inclusive one.
However some issues as data imbalance must be addressed further on.

## 7) Sequels

### Firstly We fetch Sequel Data

In [ ]:
# data comes from https://data.world/priyankad0993/sequels
df_sequels = pd.read_excel(DATA_FOLDER + 'sequels.xlsx')[["Title", "Release Date", "Year", "IMDb Rating", "Runtime (mins)", "Genres", "Num Votes", "Directors", "Movie Series", "Order"]]#,
              #dtype={'Name': str, 'Value': float}) 

df_sequels["Genres"] = df_sequels["Genres"].apply(lambda x: x.split(", "))

In [ ]:
# we need the titles in lower case
df_sequels['Title'] = df_sequels['Title'].str.lower().str.replace('[^\w\s]','')
df_data['movie_title'] = df_data['movie_title'].str.lower().str.replace('[^\w\s]','')

### we only want to keep those for which we have data

In [ ]:
df_movie_series = df_sequels.merge(df_data, left_on=['Title', "Year"], right_on=['movie_title', "Year"], how='inner')

### Now that we have the sequels and prequels we need to preprocess the data

In [ ]:
df_movie_series.shape[0]

In [ ]:
# we 
df_movie_series[~df_movie_series["box_office_revenue"].isna()].shape[0]

### We remove movies where we don't have data for at least two of the series

Sometimes the sequel came after our dataset cutoff (November 2, 2012) so we don't have data for these

In [ ]:
# We dont keep movies with only one movie in that Movie Series
df_movie_series = df_movie_series.groupby('Movie Series').filter(lambda x: len(x) >= 2)
df_movie_series.shape[0]

In [ ]:
# Grouping by "Movie Series"
grouped = df_movie_series.groupby('Movie Series')

# Creating a DataFrame for prequels (movies with the minimum Order in each group)
prequels_df = df_movie_series[df_movie_series['Order'] == grouped['Order'].transform('min')]

# Creating a DataFrame for sequels (movies where Order is not the minimum in each group)
sequels_df = df_movie_series[df_movie_series['Order'] != grouped['Order'].transform('min')]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Creating a figure with two subplots
# Combine the data for both prequels and sequels to find the common y-axis limit
combined_data = pd.concat([prequels_df["Movie Score"], sequels_df["Movie Score"]])
y_max = combined_data.max()  # Find the maximum value to set the y-axis limit


# Plotting boxplot for prequels
axes[0].boxplot(prequels_df["Movie Score"])
axes[0].set_title('Prequels Boxplot')
axes[0].set_ylabel('Movie Score (function of revenue and rating)')
axes[0].set_ylim(-2, y_max + 2) 

# Plotting boxplot for sequels
axes[1].boxplot(sequels_df["Movie Score"])
axes[1].set_title('Sequels Boxplot')
axes[1].set_ylabel('Movie Score (function of revenue and rating)')
axes[1].set_ylim(-2, y_max + 2) 

plt.tight_layout()  # Adjust layout

In [ ]:
from scipy.stats import ttest_ind, shapiro, levene

prequel_scores = prequels_df["Movie Score"]
sequel_scores = sequels_df["Movie Score"]


# Check assumptions (normality and variance equality)
# Perform Shapiro-Wilk test for normality
shapiro_sequels = shapiro(prequel_scores)
shapiro_prequels = shapiro(sequel_scores)

print(shapiro_sequels)
print(shapiro_prequels)

# Perform Levene's test for equality of variances
levene_test = levene(sequel_scores, prequel_scores)
print(levene_test)

# Perform t-test
t_stat, p_value = ttest_ind(sequel_scores, prequel_scores, equal_var=True)  # Assuming equal variances

print(t_stat, " ", p_value)

# Interpret the results
if p_value < 0.05:
    print("Reject Null Hypothesis: Significant difference between sequel and prequel scores.")
else:
    print("Failed to reject Null Hypothesis: No significant difference between sequel and prequel scores.")

# the Shapiro-Wilk tests indicate non-normality, the Levene test suggests unequal variances, 
# and the t-test suggests a significant difference in the means of the two groups. 
# These results indicate that assumptions for a t-test might not hold, and alternative approaches 
# might be necessary to compare the means of the two groups.

### We look at the differences in scores between the prequel and corresponding sequels to see if in general it increases or decreases

In [ ]:
# Create a dictionary mapping prequel scores to their series
prequel_scores_dict = prequels_df.set_index('Movie Series')['Movie Score'].to_dict()

# Function to calculate the score difference based on the series
def score_difference(row):
    prequel_score = prequel_scores_dict.get(row['Movie Series'], None)
    if prequel_score is not None:
        return row['Movie Score'] - prequel_score
    else:
        return None

# Apply the function to create a new column 'Score Difference'
sequels_df['Score Difference'] = sequels_df.apply(score_difference, axis=1)

In [ ]:
sequels_df["Score Difference"].hist()

#### Clearly in general, sequels are worst, but we'd like to see if the movies that did improve have something in common.
#### In other words we are searching for a positive change from prequel to sequels

In [ ]:
sequels_that_improved = sequels_df[sequels_df["Score Difference"] >= 0][["Title", "Release Date","IMDb Rating", "Genres", "AdjustedRevenue", "Movie Score", "Score Difference", "Good Movie"]]
sequels_that_worstened = sequels_df[sequels_df["Score Difference"] < 0][["Title", "Release Date","IMDb Rating", "Genres", "AdjustedRevenue", "Movie Score", "Score Difference", "Good Movie"]]
sequels_that_improved.head()

We can see that most of these movies are Categorized as Good Movies. Lets dig deeper

In [ ]:
# Creating pie charts for 'Good Movie' column in prequels and sequels DataFrames
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Choose contrasting shades within the 'vlag' palette
colors = [sns.color_palette('vlag')[0], sns.color_palette('vlag')[-1]]

keys = ["Bad Movies", "Good Movies"]
prequel_counts = prequels_df["Good Movie"].value_counts()
sequel_counts = sequels_df["Good Movie"].value_counts()

# declaring exploding pie 
explode = [0.1, 0] 
# define Seaborn color palette to use
  
# plotting prequel Good Movie share
axs[0].pie([prequel_counts[0], prequel_counts[1]], labels=keys, colors=colors, 
        explode=explode, autopct='%.0f%%') 
axs[0].set_title("Prequel share of Good and Bad Movies")

# Sequels
axs[1].pie([sequel_counts[0], sequel_counts[1]], labels=keys, colors=colors, 
        explode=explode, autopct='%.0f%%') 
axs[1].set_title("Sequel share of Good and Bad Movies")
plt.tight_layout()

### Analysis
We can see that much more sequels are good movies than bad movies, eventhough most of them worsten in Movie Score

As if sequels are a safe way of surfing the popularity of a previous movie; however, they don't usually result in increased popularity.

In addition, it makes sense that a few prequels are Bad Movies, because it is very risky to tackle a poor movie

### Can we find a trend in the sequels that improved?

#### Lets see if we can find one among the Genres

In [ ]:
# Explode the lists in the 'genres' column
exploded_genres = sequels_that_improved.explode('Genres')

# Get the count of each genre
genre_counts = exploded_genres['Genres'].value_counts()
genre_counts

In [ ]:
# Explode the lists in the 'genres' column
exploded_genres = sequels_that_worstened.explode('Genres')

# Get the count of each genre
genre_counts = exploded_genres['Genres'].value_counts()
genre_counts

Can't see any significant differences (note that there are more sequels that worstened which explains the differences in counts) and the data is too sparse to try and find slight distinctions between the two sets.

Deeper analysis will be performed in P3

## 8) Actors' popularity

First we need to read the actors dataset.

In [ ]:
characters_columns = ['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 'actor_dob',
                      'gender',
                      'height', 'ethnicity', 'actor_name', 'actor_age_at_release', 'freebase_character_map', '0', '1']

characters = pd.read_csv(DATA_FOLDER + 'character.metadata.tsv', sep='\t', header=None, names=characters_columns,
                         usecols=['wikipedia_movie_id', 'actor_dob', 'actor_name', 'character_name', 'release_date'])
characters.head()

In [ ]:
movies_actors = movies.copy(deep=True)
movies_actors.head()

We don't need all the columns of movies dataset for this analysis.

In [ ]:
movies_actors = movies_actors[
    ['wikipedia_movie_id', 'name', 'release_date', 'movie_score', 'adjusted_revenue', 'genres', 'year']]
movies_actors.rename(columns={'genres': 'movie_genres', 'name': 'movie_title'}, inplace=True)
movies_actors.head()

Filtering the actors dataset to only contain actors of the movies left after preprocessing.

In [ ]:
print('Number of actors before filtering: ', characters.shape[0])
characters_relevant = characters[characters['wikipedia_movie_id'].isin(movies_actors['wikipedia_movie_id'])]
characters_relevant = characters_relevant[~characters_relevant['actor_name'].isna()]
print('Number of actors after filtering: ', characters_relevant.shape[0])

We want to define actor's popularity at the time of the movie release. For this reason we want to sum `movie_score` of movies released on previous years for which actor was part of the cast.

In [ ]:
def get_actors_popularity(movies_data, char_data, lookback_movies=-1):
    popularity = []

    movies_data.sort_values(by=['year'], inplace=True)
    
    for i, movie in tqdm(movies_data.iterrows(), total=movies_data.shape[0]):
        actors = char_data[char_data['wikipedia_movie_id'] == movie['wikipedia_movie_id']]
        for j, actor in actors.iterrows():
            actor_name = actor['actor_name']

            relevant_movies = char_data[char_data['actor_name'] == actor_name]['wikipedia_movie_id']
            actor_movies = movies_data[movies_data['wikipedia_movie_id'].isin(relevant_movies)]
            
            if lookback_movies > 0:
                actor_movies = actor_movies[actor_movies['year'] < movie['year']][-lookback_movies:]
            else:
                actor_movies = actor_movies[actor_movies['year'] < movie['year']]

            popularity.append({'actor_name': actor_name, 'actor_popularity': actor_movies['movie_score'].sum(),
                                      'year': movie['year'], 'character_name': actor['character_name'],
                                      'movie_title': movie['movie_title'],
                                      'wikipedia_movie_id': movie['wikipedia_movie_id'],
                                      'movie_score': movie['movie_score']})

    return pd.DataFrame(popularity)

#### Popularity of the actors in all the previous movies they have played in
Here we are taking into account all the movies the actor has played in before the movie we are considering.

In [ ]:
try:
    actors_popularity = pd.read_csv(ADDITIONAL_FOLDER + 'actors_popularity.csv')
except:
    actors_popularity = get_actors_popularity(movies_actors, characters_relevant)
    actors_popularity.to_csv(ADDITIONAL_FOLDER + 'actors_popularity.csv', index=False)

In [ ]:
actors_popularity.sort_values(['actor_name', 'year'])

In [ ]:
grouped_popularity = actors_popularity.groupby(['movie_title', 'movie_score']).agg(
    sum_pop=('actor_popularity', 'sum')
).reset_index()
grouped_popularity.head()

In [ ]:
stats.spearmanr(grouped_popularity['movie_score'], grouped_popularity['sum_pop'])

As we can see there is a small positive correlation between the popularity of the actors and the movie score.

#### Popularity of the actors in the last 5 movies they have played in
Here we are taking into account only the last 5 movies the actor has played in before the movie we are considering.

In [ ]:
try: 
    actors_popularity = pd.read_csv(ADDITIONAL_FOLDER + 'actors_popularity_5.csv')
except:
    actors_popularity = get_actors_popularity(movies_actors, characters_relevant, 5)
    actors_popularity.to_csv(ADDITIONAL_FOLDER + 'actors_popularity_5.csv', index=False)

grouped_popularity = actors_popularity.groupby(['movie_title', 'movie_score']).agg(
    sum_pop=('actor_popularity', 'sum')
).reset_index()

In [ ]:
stats.spearmanr(grouped_popularity['movie_score'], grouped_popularity['sum_pop'])

As we can see there is a small positive correlation between the popularity of the actors and the movie score.

#### Popularity of actors vs movie score for different genres

We want to see if the correlation between the popularity of actors and the movie score is different for different genres.

In [ ]:
popular_actors_by_genres = movies_actors.copy(deep=True)

First we need to separate genres for each movie making them separate rows.

In [ ]:
movie_genres = pd.DataFrame(columns=["wikipedia_movie_id", "genre_id"])
genre_dictionary = {}
for i in range(popular_actors_by_genres.shape[0]):
    wiki_id = popular_actors_by_genres["wikipedia_movie_id"].iloc[i]
    dico = ast.literal_eval(popular_actors_by_genres["movie_genres"][i])
    data = [{'wikipedia_movie_id': wiki_id, 'genre_id': key} for key in dico.keys()]
    dataframe = pd.DataFrame(data)
    movie_genres = pd.concat([movie_genres, dataframe], axis=0)
    genre_dictionary.update(dico)

movie_genres.head()

Here is the list of all the genres.

In [ ]:
genre_dictionary.values()

Let's plot the number of movies per genre.

In [ ]:
plt.figure(figsize=(20, 10))
values = movie_genres['genre_id'].value_counts()
values.rename(index=genre_dictionary, inplace=True)
values[:50].plot(kind='bar')
plt.xlabel('Genre')
plt.ylabel('Number of movies')
plt.title('Number of movies per genre for 50 genres', fontsize=20)
plt.show()

As we can see there are some genres with a very small number of movies. We will remove them.

In [ ]:
considered_number_of_movies = 500

reduced_movie_genres = movie_genres.groupby('genre_id').filter(lambda x: len(x) > considered_number_of_movies).copy(
    deep=True)
reduced_genre_dictionary = {key: value for key, value in genre_dictionary.items() if
                            key in reduced_movie_genres['genre_id'].unique()}

In [ ]:
reduced_genre_dictionary.values()

In [ ]:
try:
    actors_popularity = pd.read_csv(ADDITIONAL_FOLDER + 'actors_popularity.csv')
except:  
    actors_popularity = get_actors_popularity(movies_actors, characters_relevant)
    actors_popularity.to_csv(ADDITIONAL_FOLDER + 'actors_popularity.csv', index=False)
    
actors_popularity.head()

We need to separate different genres into different datasets and merge them with actors popularity.

In [ ]:
datasets_by_genres = []

for genre in reduced_genre_dictionary:
    genre_movies = reduced_movie_genres[reduced_movie_genres['genre_id'] == genre]
    genre_movies = pd.merge(genre_movies, actors_popularity, on='wikipedia_movie_id')
    datasets_by_genres.append(genre_movies)

In [ ]:
grouped_datasets_by_genres = []

for df in datasets_by_genres:
    grouped_popularity = df.groupby(['movie_title', 'movie_score', 'genre_id']).agg(
        sum_pop=('actor_popularity', 'sum')
    ).reset_index()
    grouped_datasets_by_genres.append(grouped_popularity)

Let's see the correlation between the popularity of actors and the movie score for each genre.

In [ ]:
print('Correlation between sum of actor popularity and movie score for:\n')

correlations = []

for df in grouped_datasets_by_genres:
    correlations.append({'genre': reduced_genre_dictionary[df['genre_id'].iloc[0]], 'corr': stats.spearmanr(df['movie_score'], df['sum_pop'])})
    
correlations.sort(key=lambda x: x['corr'][0], reverse=True)
    
for stat in correlations:
    print(stat['genre'], ': ', stat['corr'][0])

## 9) Movie directors' popularity

## Sentiment analysis of movies endings (IN MILESTONE 3)